In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Decision Tree
We have briefly covered the decision trees in the previous chapter (*Regression Tree*).

Now, we'll explain some advanced topics.

## Splitting criteria
In a Decision Tree, the process of splitting data at each node is important. The splitting criteria finds the best feature to split the data on. In the regressor chapter we used **variance reduction** for splitting mechanism. Other common splitting criteria include **Gini Impurity** and **Entropy**.

#### Gini Impurity
This criterion measures how "impure" a node is. **Gini Impurity** represents the probability of **incorrectly classifying** a randomly chosen element if it were labeled according to the distribution of labels in the subset. The lower the Gini Impurity, the better the feature splits the data into distinct categories.

To calculate Gini impurity, use the formula:
$$ \text{Gini Impurity} = 1 - \sum{(p_i)^2}$$
where $p_i$ is the proportion of class $i$ in a node. For example, if a node has $5$ "Yes" and $5$ "No" samples, the Gini impurity is:
$$1 - (\frac{5}{10})^2 - (\frac{5}{10})^2 = 0.5$$
indicating a $50$% chance of misclassification. A perfectly pure node with all "Yes" samples has a Gini impurity of:
$$1 - (1)^2 - (0)^2= 0$$

In general the formula is:
$$Gini(p_1, p_2) = 1 - p_1^2 - p_2^2$$

<center><img src="img/gini_impurity.png" alt="Gini impurity" width="800" height="567" /></center>
<p style="text-align: center; font-size: small;"><i><b>Figure 1.</b> 0.394 is high impurity. It says there is a mix of classes in that tree and its subtrees. </i></p>


In [2]:
def gini_index(Y: np.ndarray):
    '''
    Calculate the GINI INDEX for a given target variable.

    Args:
        Y (np.ndarray): The column vector of the target variable.

    Returns:
        float: The Gini index.
    '''
    # Extract all unique classes in the target variable
    class_labels = np.unique(Y)
    gini = 0
    # For each class, calculate the proportion of samples that belong to that class
    for cls in class_labels:
        # Calculate the proportion of samples that belong to the current class to all samples in the dataset
        p_cls = len(Y[Y == cls]) / len(Y)
        # Sum the squared proportions of each class
        gini += p_cls**2
    # Return 1 minus the sum of the squared proportions of each class (as per the formula)
    return 1 - gini

#### Entropy
This measures the **amount of uncertainty or randomness** in the data. The tree tries to reduce the entropy by splitting the data on features that provide the most information about the target variable.
These criteria help decide which features are useful for making the best split at each decision point in the tree.

* If a dataset contains examples from only *one class*, its **entropy is zero**, indicating complete purity.
* If the dataset is **evenly split** between classes, its **entropy is at its maximum** which is 1, indicating maximum randomness.

The formula for entropy (H) for a binary classification problem is:
$$H(p_1, p_2) = - p_1 log_2(p_1) - p_2 log_2(p_2)$$
where $p_1$ and $p_2$ are the probabilities of two classes.


<center><img src="img/entropy_1.png" alt="How is entropy measured?" width="900" height="491" /></center>
<p style="text-align: center; font-size: small;"><i><b>Figure 2.</b> The entropy of a binary classification problem is in [0, 1] range</i></p>


In [3]:
def entropy(Y: np.ndarray):
    '''
    Calculate the ENTROPY for a given target variable.

    Args:
        Y (np.ndarray): The column vector of the target variable.

    Returns:
        float: The entropy.
    '''

    class_labels = np.unique(Y)
    entropy = 0
    for cls in class_labels:
        p_cls = len(Y[Y == cls]) / len(Y)
        entropy += -p_cls * np.log2(p_cls) # It is almost the same as Gini impurity (extra logarithmic calculation is added)
    return entropy

#### Conclusion
As you can see the formulas are quite similar. Thus, they lead to similar results. That's why people mostly prefer to use **Gini**, since they want to avoid the logarithmic calculation (which is relatively slow operation).

|  | Gini | Entropy |
| --- | --- | --- |
| **Measure** | Probability of misclassifying a random input | The amount of uncertainty (randomness) in a set |
| **Range** | $[0, 0.5]$ | $[0, log2(C)]$, where $C$ is the number of classes. For binary classification it is $[0, 1]$ |
| **Interpretation** | The expected error rate in a classifier. | The average amount of information needed to specify the class of an instance. |
| **Sensitivity** | It is sensitive to the distribution of classes in a set. | It is sensitive to the number of classes. |
| **Bias** | It has a bias toward selecting splits that result in a more balanced distribution of classes. | It has a bias toward selecting splits that result in a higher reduction of uncertainty. |

## Pruning

* Pruning is an important technique used to **prevent overfitting** in Decision Trees. Overfitting occurs when a tree becomes too deep and starts to memorize the training data rather than learning general patterns. This leads to poor performance on new, unseen data.
* This technique reduces the complexity of the tree by **removing branches** that have **little predictive power**. It improves model performance by helping the tree generalize better to new data. It also makes the model simpler and faster to deploy.
* It is useful when a Decision Tree is too deep and starts to capture noise in the data.

There are two main types of decision tree pruning: **Pre-Pruning** and **Post-Pruning**.

#### Pre-Pruning (Early Stopping)
Sometimes, the growth of the decision tree can be stopped before it gets too complex, this is called pre-pruning. It is important to prevent the overfitting of the training data, which results in a poor performance when exposed to new data.

Some common pre-pruning techniques include:

* **Maximum Depth**: It limits the maximum level of depth in a decision tree.
* **Minimum Samples per Leaf**: Set a minimum threshold for the number of samples in each leaf node.
* **Minimum Samples per Split**: Specify the minimal number of samples needed to break up a node.
* **Maximum Features**: Restrict the quantity of features considered for splitting.

#### Post-Pruning (Reducing Nodes)
After the­ tree is fully grown, post-pruning involves re­moving branches or nodes to improve the­ model's ability to generalize­. Some common post-pruning techniques include­:

* **Cost-Complexity Pruning (CCP)**: This method assigns a price to each subtre­e primarily based on its *accuracy* and *complexity*, the­n selects the subtre­e with the *lowest fee*.
* **Re­duced Error Pruning**: Removes branche­s that do not significantly affect the overall accuracy.
* **Minimum Impurity De­crease**: Prunes node­s if the decrease­ in impurity (Gini impurity or entropy) is beneath a ce­rtain threshold.
* **Minimum Leaf Size**: Re­moves leaf nodes with fe­wer samples than a specifie­d threshold.

Post-pruning simplifies the tre­e while preserving its Accuracy.

# Classification Tree

It is the same as the **regression tree**. The **only difference** is that for information gain we will not be using **variance_reduction** method, but **Gini index** instead.

In [4]:
def information_gain(parent: np.ndarray, l_child: np.ndarray , r_child: np.ndarray):
    ''' 
    In regression tree variance reduction is used. In classification tasks we use 
    Gini Index or Entropy to calculate the information gain.
    '''
    weight_l = len(l_child) / len(parent)
    weight_r = len(r_child) / len(parent)
    return gini_index(parent) - (weight_l*gini_index(l_child) + weight_r*gini_index(r_child))

## Implementation

#### Node

The node implementation is the same as the one in the **regression tree**. However, instead of `var_red` we have `info_gain` now.

In [5]:
class Node():
    def __init__(self,
    feature_index: int = None,
    threshold: float = None,
    info_gain: float = None,
    left: 'Node' = None,
    right: 'Node' = None,
    value: float = None):
        '''
        Constructor for the Node class.

        Args:
            feature_index (int):    The index of the feature to split on.
            threshold (float):      The threshold value to split on.
            info_gain (float):      The information gain achieved by the split.
            left (Node):            The left child node.
            right (Node):           The right child node.
            value (float):          The value of the node (in case of a leaf node).
        ''' 
        # for decision node
        self.feature_index = feature_index
        self.threshold = threshold
        self.info_gain = info_gain
        self.left = left
        self.right = right
        
        # for leaf node
        self.value = value

#### Tree

Now, let's build the tree. The magic here happens in `get_best_split` method. There we evaluate candidate splits and rank them by gini index. The one with highest gini becomes a new decision node in our tree. Then we go deeper, until we hit a tree `max_depth` limit.

In [ ]:
class ClassificationTree():
    def __init__(self, min_samples_split: int = 2, max_depth: int = 3):
        '''
        Constructor for the RegressionTree class.

        Args:
            min_samples_split (int): The minimum number of samples required to split an internal node.
            max_depth (int):         The maximum depth of the tree.
        '''
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.root = None

    def build_tree(self, dataset: np.ndarray, curr_depth: int = 0) -> Node:
        '''
        Recrusively build the decision tree.

        Args:
            dataset (np.ndarray):   The dataset to build the tree on.
            curr_depth (int):       The current depth of the tree.

        Returns:
            Node: The root node of the tree.
        '''
        # Split the dataset into features and target variable
        X, Y = dataset[:,:-1], dataset[:,-1]
        num_samples, num_features = np.shape(X)
        best_split = {}

        # Split until stopping conditions are met (pre-pruning criteria):
        # 1. Minimum number of samples to split an internal node
        # 2. Maximum depth of the tree
        if num_samples >= self.min_samples_split and curr_depth <= self.max_depth:
            # find the best split
            best_split = self.get_best_split(dataset, num_features)
            # check if information gain is positive
            if best_split["info_gain"] > 0:
                # Recursively build the left subtree
                left_subtree = self.build_tree(best_split["dataset_left"], curr_depth+1)
                # Recursively build the right subtree
                right_subtree = self.build_tree(best_split["dataset_right"], curr_depth+1)
                # CASE 1: Return a new decision node (root node for the two new subtrees)
                return Node(best_split["feature_index"],
                            best_split["threshold"],
                            best_split["info_gain"],
                            left_subtree, 
                            right_subtree)
        
        # CASE 2: Otherwise, compute and return a leaf node.
        leaf_value = self.calculate_leaf_value(Y)
        return Node(value=leaf_value)

    def get_best_split(self, 
            dataset: np.ndarray,
            num_features: int) -> dict:
        '''
        Find the best split for the dataset. This happens by building a candidate split for each feature and each value of this feature.
        So the complexity of this method is O(m x u), where m is the number of features and u is the avg number of unique values within a feature.
        Wa rank the candidate splits by `information gain` (gini indexn) and always pick the one with highest `information gain`.

        Args:
            dataset (np.ndarray):   The dataset to find the best split on.
            num_features (int):     The number of features in the dataset.

        Returns:
            dict: The best split.
        '''
        # Dictionary to store the best split
        best_split = {}
        # Start with an extremely small number.
        max_info_gain = -float("inf")

        # Loop over all the features
        for feature_index in range(num_features):
            feature_values = dataset[:, feature_index]
            # List of candidate thresholds - unique values of the feature.
            threshold_candidates = np.unique(feature_values)
            # Loop over all the feature values present in the data
            for threshold_candidate in threshold_candidates:
                # Build a candidate split.
                dataset_left, dataset_right = self.split(dataset, feature_index, threshold_candidate)
                # If childs are not empty.
                if len(dataset_left) > 0 and len(dataset_right) > 0:
                    y, left_y, right_y = dataset[:, -1], dataset_left[:, -1], dataset_right[:, -1]
                    # Then, compute the information gain of the candidate split.
                    # NB: THIS IS THE ONLY DIFFERENCE WITH THE REGRESSION TREE
                    curr_info_gain = information_gain(y, left_y, right_y)
                    # And update the best split if needed. The best split is the one 
                    # with the highest information gain (gini index).
                    if curr_info_gain > max_info_gain:
                        best_split["feature_index"] = feature_index
                        best_split["threshold"]     = threshold_candidate
                        best_split["dataset_left"]  = dataset_left
                        best_split["dataset_right"] = dataset_right
                        best_split["info_gain"]     = curr_info_gain
                        max_info_gain = curr_info_gain

        # Return the best split (the one with the highest information gain).
        return best_split

    def split(self, 
            dataset: np.ndarray, 
            feature_index: int, 
            threshold: float) -> tuple[np.ndarray, np.ndarray]:
        ''' 
        Split the dataset into two subsets (candidate split) based on the feature values and threshold.

        Args:
            dataset (np.ndarray):   The dataset to split.
            feature_index (int):    The index of the feature to split on.
            threshold (float):      The threshold value to split on.

        Returns:
            tuple: The candidate split (two subsets of the dataset).
        '''
        dataset_left = np.array([row for row in dataset if row[feature_index] <= threshold])
        dataset_right = np.array([row for row in dataset if row[feature_index] > threshold])
        return dataset_left, dataset_right

    def calculate_leaf_value(self, Y: np.ndarray) -> float:
        ''' 
        Compute the leaf node value. Basically, the average of the target variable.

        Args:
            Y (np.ndarray): The target variable.
        '''
        val = np.mean(Y)
        return val

    def print_tree(self, tree = None, indent = " "):
        ''' 
        Recursively print the decision tree.

        Args:
            tree (Node): The root node of the tree.
            indent (str): The indentation string.
        '''
        # CASE 1: We are at root level
        if not tree:
            print("X_0 is yrs_of_XP, X_1 is education_level")
            print("------")
            tree = self.root

        # CASE 2: We are at leaf level
        if tree.value is not None:
            print(f"{tree.value:.0f}")

        # CASE 3: We are at internal node level
        else:
            info_gain_str = f"{tree.info_gain:.3f}"
            print("X_"+str(tree.feature_index), "<=", tree.threshold, "? Info Gain: ", info_gain_str)
            print("%sleft:" % (indent), end="")
            self.print_tree(tree.left, indent + indent)
            print("%sright:" % (indent), end="")
            self.print_tree(tree.right, indent + indent)

    def fit(self, X: np.ndarray, Y: np.ndarray):
        ''' 
        Train the decision tree.

        Args:
            X (np.ndarray): The input features.
            Y (np.ndarray): The target variable.
        '''

        dataset = np.concatenate((X, Y), axis=1)
        self.root = self.build_tree(dataset)

    def make_prediction(self, x: np.ndarray, tree: Node) -> float:
        '''
        Recursively predict the target variable for a new dataset.

        Args:
            x (np.ndarray): A data point from the dataset.
            tree (Node):    The root node of the tree.
        '''
        # CASE 1: Leaf node
        if tree.value is not None: 
            return tree.value

        # CASE 2: Decision node
        feature_val = x[tree.feature_index]
        if feature_val <= tree.threshold:
            # CASE 2.1: Go down the left child
            return self.make_prediction(x, tree.left)
        else:
            # CASE 2.2: Go down the right child
            return self.make_prediction(x, tree.right)

    def predict(self, X: np.ndarray) -> np.ndarray:
        ''' 
        Predict the target variable for a new dataset.

        Args:
            X (np.ndarray): All the input features from the dataset.

        Returns:
            np.ndarray:     The predicted target variable.
        '''
        # For each data point in the dataset, make a prediction.
        predictions = [self.make_prediction(x, self.root) for x in X]
        return predictions

# Usage
#### School dropout prediction
Now, we'll use our **Classification Tree** to predict school dropouts (pupils who fall out of school before completing their educational program). For that sake, we will use a popular [students dataset](https://www.kaggle.com/datasets/mahwiz/students-dropout-and-academic-success-dataset).

In [7]:
# Load data
students_df = pd.read_csv("data/students_dataset.csv.gz", compression="gzip")
print(f"Shape of the dataset: {students_df.shape}")
students_df.head()

Shape of the dataset: (4423, 37)


,Marital status,Application mode,Application order,Course,Daytime/evening attendance\t,Previous qualification,Previous qualification (grade),Nacionality,Mother's qualification,Father's qualification,...,Curricular units 2nd sem (credited),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Unemployment rate,Inflation rate,GDP,Target
0,1,17,5,171,1,1,122.0,1,19,12,...,0,0,0,0,0.000000,0,10.8,1.4,1.74,Dropout
1,1,15,1,9254,1,1,160.0,1,1,3,...,0,6,6,6,13.666667,0,13.9,-0.3,0.79,Graduate
2,1,1,5,9070,1,1,122.0,1,37,37,...,0,6,0,0,0.000000,0,10.8,1.4,1.74,Dropout
3,1,17,2,9773,1,1,122.0,1,38,37,...,0,6,10,5,12.400000,0,9.4,-0.8,-3.12,Graduate
4,2,39,1,8014,0,1,100.0,1,37,38,...,0,6,6,6,13.000000,0,13.9,-0.3,0.79,Graduate


In this dataset we have 4423 student records. For each one of them we have information about the socio-economic status of the student and her parents. Furthermore, we have the target variable: has the student `Graduated` or `Dropped out` of school.

#### Preprocess data

We also have 800 students in `Enrolled` state. We will simply discard them for the sake of simplicity.


In [8]:
# Drop `Enrolled` status (we will operate only with Dropouts and Graduates)
students_df = students_df[ (students_df["Target"] != 'Enrolled')]
print(f"Shape of the dataset: {students_df.shape}")

Shape of the dataset: (3629, 37)


Before we start our training we need to convert the labels (`Graduate` and `Dropout` into numeric values - $0$ and $1$).

In [9]:
# Convert the target variable to binary class (0 vs 1)
students_df.Target = students_df.Target.replace({"Graduate": 0, "Dropout": 1})
students_df.head()

C:\Users\mdzho\AppData\Local\Temp\ipykernel_17180\3877963097.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  students_df.Target = students_df.Target.replace({"Graduate": 0, "Dropout": 1})


,Marital status,Application mode,Application order,Course,Daytime/evening attendance\t,Previous qualification,Previous qualification (grade),Nacionality,Mother's qualification,Father's qualification,...,Curricular units 2nd sem (credited),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Unemployment rate,Inflation rate,GDP,Target
0,1,17,5,171,1,1,122.0,1,19,12,...,0,0,0,0,0.000000,0,10.8,1.4,1.74,1
1,1,15,1,9254,1,1,160.0,1,1,3,...,0,6,6,6,13.666667,0,13.9,-0.3,0.79,0
2,1,1,5,9070,1,1,122.0,1,37,37,...,0,6,0,0,0.000000,0,10.8,1.4,1.74,1
3,1,17,2,9773,1,1,122.0,1,38,37,...,0,6,10,5,12.400000,0,9.4,-0.8,-3.12,0
4,2,39,1,8014,0,1,100.0,1,37,38,...,0,6,6,6,13.000000,0,13.9,-0.3,0.79,0


In [10]:
students_df.dtypes

Marital status                                      int64
Application mode                                    int64
Application order                                   int64
Course                                              int64
Daytime/evening attendance\t                        int64
Previous qualification                              int64
Previous qualification (grade)                    float64
Nacionality                                         int64
Mother's qualification                              int64
Father's qualification                              int64
Mother's occupation                                 int64
Father's occupation                                 int64
Admission grade                                   float64
Displaced                                           int64
Educational special needs                           int64
Debtor                                              int64
Tuition fees up to date                             int64
Gender        

#### Train-Test split
We will also need a simple method for shuffling and splitting the data into `train_set` and `test_set`. Latter is used for measuring the model's accuracy.

In [11]:
def train_test_split(
    df: pd.DataFrame, 
    test_size: float = 0.2) -> tuple[pd.DataFrame, pd.DataFrame]:
    '''
    Split the dataset into training and testing sets.

    Args:
        X (np.ndarray): The input features.
        Y (np.ndarray): The target variable.
        test_size (float): The size of the test set.

    Returns:
        tuple[pd.DataFrame, pd.DataFrame]: The training and testing sets.
    '''
    # Shuffle the dataset
    df = df.sample(frac=1).reset_index(drop=True)
    # Calculate the number of rows in the test set
    n_test_rows = int(len(df) * test_size)
    # Split the dataset
    train_df = df.iloc[:-n_test_rows]
    test_df = df.iloc[-n_test_rows:]
    return train_df, test_df


train_df, test_df = train_test_split(students_df)
X_train, Y_train = train_df.iloc[:, :-1].values, train_df.iloc[:, -1].values.reshape(-1,1)
X_test, Y_test = test_df.iloc[:, :-1].values, test_df.iloc[:, -1].values.reshape(-1,1)

print(X_train.shape, Y_train.shape)

(2904, 36) (2904, 1)


#### Train the model

And now... finally... let's train our model and see if it builds the tree (pick good splits) in a way that will be predicting future dropout students.

In [12]:
regression_tree = ClassificationTree(min_samples_split=3, max_depth=3)
regression_tree.fit(X_train, Y_train)
regression_tree.print_tree()

X_0 is yrs_of_XP, X_1 is education_level
------
X_30 <= 3.0 ? Info Gain:  0.243
 left:X_22 <= 0.0 ? Info Gain:  0.053
  left:X_16 <= 0.0 ? Info Gain:  0.051
    left:X_33 <= 12.4 ? Info Gain:  0.040
        left:1
        right:1
    right:X_12 <= 135.8 ? Info Gain:  0.049
        left:1
        right:0
  right:X_24 <= 5.0 ? Info Gain:  0.011
    left:X_30 <= 2.0 ? Info Gain:  0.003
        left:1
        right:1
    right:X_30 <= 2.0 ? Info Gain:  0.114
        left:1
        right:0
 right:X_16 <= 0.0 ? Info Gain:  0.047
  left:X_6 <= 143.0 ? Info Gain:  0.073
    left:X_18 <= 0.0 ? Info Gain:  0.050
        left:1
        right:0
    right:X_1 <= 39.0 ? Info Gain:  0.333
        left:0
        right:1
  right:X_30 <= 4.0 ? Info Gain:  0.010
    left:X_29 <= 9.0 ? Info Gain:  0.039
        left:0
        right:1
    right:X_23 <= 11.0 ? Info Gain:  0.005
        left:0
        right:0


#### Measure accuracy

The model predicts float values betwen $0$ and $1$. We will pick $0.5$ for a threshold and every value above this will be considered as $1$ (`Dropout`) and every value below will be considered as $0$ (`Graduate`).

In [13]:
Y_pred = regression_tree.predict(X_test) 
correct_predictions = 0
for idx in range(len(X_test)):
    prediction = Y_pred[idx]
    # Convert to 0 and 1 (0 = Graduate, 1 = Dropout)
    prediction = 1 if prediction > 0.5 else 0
    actual = Y_test[idx][0]
    if prediction == actual:
        correct_predictions += 1

print(f"\nAccuracy: {correct_predictions/len(X_test)*100:.0f}%")


Accuracy: 90%
